In [1]:
import pandas as pd
import tkinter as tk
from datetime import datetime
from pandastable import Table, TableModel
from tkinter import scrolledtext
import serial
import time

In [11]:
studentRecord = pd.read_csv("Class Lists.csv")
electronics = pd.read_csv('PowerElectronics.csv')
control = pd.read_csv('ControlSystem.csv')
system = pd.read_csv('PowerSystem.csv')

In [3]:
########################## CODE TO FIND THE RFID FOR A GIVEN SID  ######################################

# def find_rfid_for_sid(df, sid):
#     """Find the RFID value for a given SID in a Pandas DataFrame."""
#     matching_row = df[df['SID'] == sid]

#     if not matching_row.empty:
#         rfid_value = matching_row.iloc[0]['RFID']  # Or matching_row['RFID'].values[0]
#         return rfid_value
#     else:
#         return None  # Return None if the SID is not found in the DataFrame


# # Find the RFID value for a given SID
# sid_to_find = 21104002
# rfid = find_rfid_for_sid(df, sid_to_find)


In [4]:
########################## STUDENT INFO  ######################################

def StudentInfo(df, rfid):
    """Find the SID value for a given RFID in a Pandas DataFrame."""
    matching_row = df[df['RFID'] == rfid]

    if not matching_row.empty:
        sid_value = str(matching_row.iloc[0]['SID'])  # Or matching_row['RFID'].values[0]
        name_value = str(matching_row.iloc[0]['NAME'])  # Or matching_row['RFID'].values[0]
        print(f"The RFID :  {rfid}  belongs to  {name_value}, SID: {sid_value}")
        return (sid_value, name_value)
    else:
        return (str(00000000), 'I_am a ghost')


# # Find the RFID value for a given SID
# rrff = '60 B5 AB 14'
# (new_SID , new_NAME) = StudentInfo(studentRecord, rrff)
# print(new_SID)
# print(new_NAME)

In [5]:
########################## NEW COLUMN ######################################

def create_new_column(df, column_name):
    """Create a new column in a Pandas DataFrame."""
    df[column_name] = 0  


In [6]:
########################## CODE TO ADD ATTANDENCE  ######################################
def add_value_to_column_sid_wise(df, column_name, sid_value, value_to_add):

    """Add a value to a specific column in a Pandas DataFrame based on a matching SID."""
    
    if column_name not in df.columns:
        print("Column does not exist.")
        return

    df.loc[df['SID'] == sid_value, column_name] = value_to_add


# add_value_to_column_sid_wise(studentRecord, 'RFID', 21104002, 'yoyoyo')
# studentRecord


In [7]:
# ########################## CODE TO FIND A VALUE IN DB  ######################################
# def val_in_DB(df, column_name, sid_value):

#     """Add a value to a specific column in a Pandas DataFrame based on a matching SID."""
    
#     if column_name not in df.columns:
#         print("Column does not exist.")
#         return

#     value = df.loc[df['SID'] == sid_value, column_name]
#     return str(value)


# k = val_in_DB(studentRecord, 'NAME', 21104002)
# k
# studentRecord


In [7]:
dtt = datetime.now().strftime("%d %b, %a")
dtt

'31 Oct, Tue'

In [8]:
std = 'Naheed'
sid = 21104002

create_new_column(electronics, datetime.now().strftime("%d %b, %a"))

add_value_to_column_sid_wise(electronics, 'Lecture-8', sid_value=sid , value_to_add= 1)
electronics

Column does not exist.


,SID,NAME,"31 Oct, Tue"
0,21104001,Kashyap,0
1,21104002,Naheed,0
2,21104003,Abhinav,0
3,21104004,Siddhant,0
4,21104005,Aakshita,0
...,...,...,...
58,21104131,Rohan,0
59,21104132,Harshit,0
60,21104133,Rounak,0
61,21104135,Manya,0


In [9]:
add_value_to_column_sid_wise(electronics, 'Lecture-8', sid_value=21104003 , value_to_add= 1)
electronics

Column does not exist.


,SID,NAME,"31 Oct, Tue"
0,21104001,Kashyap,0
1,21104002,Naheed,0
2,21104003,Abhinav,0
3,21104004,Siddhant,0
4,21104005,Aakshita,0
...,...,...,...
58,21104131,Rohan,0
59,21104132,Harshit,0
60,21104133,Rounak,0
61,21104135,Manya,0


In [13]:
def add_to_status(text):
    status.configure(state='normal')
    status.insert(tk.INSERT , text + '\n')
    status.configure(state='disabled')
    root.update()

# def runningAtt(ser = serial.Serial('COM4', 115200), att=None, serial_port = 'COM4', baud_rate = 9600):
def runningAtt():
    print('statingATTT')
    add_to_status('Starting attandence')
    try:
        serial_port = 'COM4'  # Change this to the correct port for your system
        baud_rate = 115200
        
        ser = serial.Serial(serial_port, baud_rate)
        line = ''
        
        while (line != 'C3 13 F5 FA'):

            line = ser.readline().decode('utf-8').strip()
            # time.sleep(3)
            
            if (line == 'C3 13 F5 FA'):
                print('exiting')
                line = ''
                add_to_status('Attendance is complete')
                # ser.close()
                return
        
            (new_SID , new_NAME) = StudentInfo(studentRecord, line)
            print(f"SID:  {new_SID},  NAME:  {new_NAME}")
            add_to_status(f"SID:  {new_SID},  NAME:  {new_NAME}")
            lcdDisplay = str(new_SID + "*" + new_NAME)
            add_to_status(lcdDisplay)
            # ser.write(lcdDisplay)
        


    except serial.SerialException:
        print('Your ESP32 is not connected')
        add_to_status(f"Failed to open {serial_port}. Make sure the ESP32 is connected and the port is correct.")
    except KeyboardInterrupt:
        print("Serial monitoring stopped.")
    finally:
        print('att over')
        # if 'ser' in locals():
        #     ser.close()

def stopAtt():
    att = 2



def update_time_date_labels():
    current_time = datetime.now().strftime("%H:%M:%S")
    current_date = datetime.now().strftime("%d - %m - %Y")
    time_label.config(text="Time: " + current_time)
    date_label.config(text="Date: " + current_date)
    root.after(1000, update_time_date_labels)  # Update every second

def subject_selected(event):
    
    showingSubject = sub.get()   
    subject_label.config(text="Subject: " + showingSubject)
    print(f"Showing {sub.get()} Attandence Record")
    show_db()

def show_db():
    
    if (sub.get() == 'CLass_List'):
        subject_label.config(text="Showing: Class_List")
        # pt.updateModel()
        pt.show()
        # pt.redraw()
        # root.update()
        
    
    if (sub.get() == 'Power Electronics'):
        # root.update()
        # pt_PE.updateModel()
        pt_PE.show()
        # pt_PE.redraw()
    
    if (sub.get() == 'Control System'):
        # pt_control.updateModel()
        pt_control.show()
        # pt_control.redraw()
        # root.update()
    
    if (sub.get() == 'Power System'):
        pt_system.show()
        # pt_system.redraw()
        # pt_system.updateModel()
        # root.update()

    

#################################Create the main window #######################
root = tk.Tk()
root.geometry("1200x800")
# root.attributes('-fullscreen', True)
root.title("RFID Attendance System")

################################# TOP_L FRAME #######################
top_L = tk.Frame(root)
top_L.grid(row=0, column=0, pady=20, padx=20)

heading_label = tk.Label(top_L, text="Electrical Department", font=("Helvetica", 24))
date_label = tk.Label(top_L, text="Current Date: ", font=("Helvetica", 12))
time_label = tk.Label(top_L, text="Current Time: ", font=("Helvetica", 12))

heading_label.grid()
date_label.grid()
time_label.grid()


################################# TOP_R FRAME #######################
top_R = tk.Frame(root)
top_R.grid(row=0, column=1, pady=20, padx=20)

sub = tk.StringVar()
curr_sub = tk.OptionMenu(top_R, sub ,'CLass_List', 'Power Electronics', 'Control System', 'Power System', command = subject_selected)
subject_label = tk.Label(top_R, text="Subject :", font=("Helvetica", 12))
# att = None
start_att = tk.Button(top_R, text='Start\nattandence', command=runningAtt)
# stop_att = tk.Button(top_R, text='Stop\nattandence', command= stopAtt)

curr_sub.grid()
subject_label.grid()

################################# DATABASE FRAME #######################
middle1 = tk.Frame(root)
middle2 = tk.Frame(root)
middle3 = tk.Frame(root)
middle4 = tk.Frame(root)

middle1.grid(row=1, column=0, pady=20, padx=20)
middle2.grid(row=1, column=0, pady=20, padx=20)
middle3.grid(row=1, column=0, pady=20, padx=20)
middle4.grid(row=1, column=0, pady=20, padx=20)

middle1.table = pt = Table(middle1, dataframe=studentRecord, showtoolbar=False, showstatusbar=False ,editable=False, height = 550, width = 700)
middle2.table = pt_PE = Table(middle2, dataframe=electronics, showtoolbar=False, showstatusbar=False ,editable=False, height = 550, width = 700)
middle3.table = pt_control = Table(middle3, dataframe=control, showtoolbar=False, showstatusbar=False ,editable=False, height = 550, width = 700)
middle4.table = pt_system = Table(middle4, dataframe=system, showtoolbar=False, showstatusbar=False ,editable=False, height = 550, width = 700)
pt.show()
# show_db()


################################# STATUS FRAME #######################
bottom = tk.Frame(root)
bottom.grid(row=1, column=1, pady=20, padx=20)

status = scrolledtext.ScrolledText(bottom, wrap = tk.WORD, width =60, height = 25, font = ("Times New Roman",12))
status.grid()

# ################################# TOP_R FRAME #######################
# top_R = tk.Frame(root)
# top_R.grid(row=0, column=1, pady=20, padx=20)

start_att.grid(row=0, column=3, padx=70)
# stop_att.grid(row=0, column=5, padx=40)

update_time_date_labels()
root.mainloop()


statingATTT
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  F2 AA 41 54  belongs to  Rasneet, SID: 21104010
SID:  21104010,  NAME:  Rasneet
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID: 21104004
SID:  21104004,  NAME:  Siddhant
The RFID :  E2 DF F1 54  belongs to  Siddhant, SID